# Grupo 5 - Word2Vec + RNN


## Sumário

- **1. Setup**

- **2. Carregamento e Preparação dos Dados**
  - **2.1 Carregamento dos dados**
  - **2.2 Preparação dos dados**
  - **2.3 Divisão dos dados**
  - **2.3 Divisão dos dados**
  - **2.4 Tokenização**
  - **2.5 Normalização**
  - **2.6 Categorizar Labels**

- **3. Word2Vec**

- **4. Rede Neural Classificação**
  - **4.1 Definição da rede neural**
  - **4.2 Treinamento da rede neural**

- **5. Resultados**
  - **5.1 Avaliação do modelo**
  - **5.2 Gráfico da variação da acurácia e loss de acordo com as épocas**
  - **5.3 Gráfico para comparação das métricas Recall X Precision X F1-Score**

- **6. Performance**


# 1. Setup

Seção para realizar a prepração do ambiente de desenvolvimento, realizando a conexão com o Google Drive e importando as bibliotecas necessárias.



In [1]:
# Montagem do drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Importar bibliotecas
import time
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout, Bidirectional, GlobalMaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping

from gensim.models import KeyedVectors


# 2. Carregamento e Preparação dos Dados

Nesta seção é feito o carregamento da base de dados que está salva no drive compartilhado da turma, e fazendo todo o pré processamento dos dados, como separar em treino e teste, tokenizar, normalizar os paddings para utilizar no Word2Vector e transformando para número as intenções.

## 2.1 Carregamento dos Dados

In [3]:
# Carregar os dados
PATH = '/content/drive/Shareddrives/Grupo_05/base_aug_20240918_v1.xlsx'
data = pd.read_excel(PATH)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Exibir as primeiras linhas do dataset
data = data[['Intencao', 'Texto']]
data.head()

,Intencao,Texto
0,Acesso a conta,Atualizei os site e agora como acessar minha c...
1,Acesso a conta,Boa tarde Eu gostaria usar o aplicativo da bra...
2,Acesso a conta,Boa tarde Por gentileza alguém pode me ajudar ...
3,Acesso a conta,Boa tarde Jessica!Consegui logar minha conta!M...
4,Acesso a conta,Boa tarde estou tendo problema com o pin eu nã...


## 2.2 Preparação dos dados

In [18]:
print(data.columns)

Index(['Unnamed: 0', 'Intencao', 'Texto'], dtype='object')


In [20]:
perguntas = data[['Intencao', 'Texto']].copy()

In [21]:
# Carregar os dados
PATH = '/content/drive/Shareddrives/Grupo_05/base_aug_20240918_v1.xlsx'
data = pd.read_excel(PATH)

# Criar um novo DataFrame com as colunas desejadas, mantendo o original intacto
data_intencao_texto = data[['Intencao', 'Texto']]

# Exibir as primeiras linhas do novo dataset
data_intencao_texto.head()

# Agora, use o DataFrame original 'data' para acessar 'Pergunta_Lematizada' e 'Resposta_Lematizada'
# e crie os DataFrames 'perguntas' e 'respostas' primeiro
perguntas = data[['Intencao', 'Texto']].copy()
respostas = data[['Intencao', 'Texto']].copy()

# Depois de definir 'perguntas' e 'respostas', renomeie as colunas
perguntas = perguntas.rename(columns={'Intencao': 'Texto'})
respostas = respostas.rename(columns={'Intencao': 'Texto'})

base = pd.concat([perguntas,respostas])
base.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6894 entries, 0 to 3446
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Texto   6892 non-null   object
 1   Texto   6892 non-null   object
dtypes: object(2)
memory usage: 161.6+ KB


In [22]:
perguntas = data[['Intencao', 'Texto']].copy()
respostas = data[['Intencao', 'Texto']].copy()
perguntas = perguntas.rename(columns={'Intencao': 'Texto'})
respostas = respostas.rename(columns={'Intencao': 'Texto'})
base = pd.concat([perguntas,respostas])
base.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6894 entries, 0 to 3446
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Texto   6892 non-null   object
 1   Texto   6892 non-null   object
dtypes: object(2)
memory usage: 161.6+ KB


In [23]:
base = data
base = base.dropna()
base = base.drop_duplicates(subset='Texto')
base.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3445 entries, 0 to 3446
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  3445 non-null   int64 
 1   Intencao    3445 non-null   object
 2   Texto       3445 non-null   object
dtypes: int64(1), object(2)
memory usage: 107.7+ KB


## 2.3 Divisão dos dados

In [24]:
# Definição de labels e dados que serão utilizados para o treinamento
quests = base['Texto'].astype(str).values
labels = base['Intencao'].values

In [25]:
# Dividir os dados em conjuntos de treinamento e teste sendo 75% e 25%
train_data, test_data, train_labels, test_labels = train_test_split(quests, labels, test_size=0.25, random_state=42)

## 2.4 Tokenização

In [26]:
# Tokenização das perguntas com um token OOV para palavras desconhecidas
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(train_data)  # Tokeniza o texto
train_sequences = tokenizer.texts_to_sequences(train_data)
test_sequences = tokenizer.texts_to_sequences(test_data)

In [27]:
# Visualizando as sequências geradas
print("Sequências de treino:", train_sequences)
print("Sequências de teste:", test_sequences)

Sequências de treino: [[142, 1550, 20, 171], [8, 4, 513, 48, 223, 2, 4, 122, 958, 821, 272, 4, 17, 5, 513, 687, 273, 129, 4, 362, 82, 40, 959, 582, 883], [15, 13, 9, 33, 75, 27, 239, 129, 559, 629, 176, 1322], [498, 182, 480, 884, 514, 1147], [481, 1323, 9, 515, 375, 213, 239, 37, 12, 240, 2, 241, 1324, 1325, 172, 2, 22, 14, 67, 123, 80, 2, 96, 314, 7, 14, 583, 172, 196, 197, 529, 32, 11, 530, 584, 56, 241, 1326, 463, 355, 464, 57, 452, 22, 14, 100, 499, 41, 389, 390, 560, 32, 650, 240, 284, 338, 531, 391, 32, 376, 516, 5, 607, 339, 89, 47, 10, 96, 8, 198, 1035, 120, 228, 25, 561, 196, 197, 37, 416, 119, 28, 42, 35, 40, 43], [16, 292, 299, 45, 36, 21, 960, 103, 300, 213, 608], [26, 10, 7, 6, 4, 77, 20, 44, 2, 53, 24, 44, 5, 23, 50, 64, 730], [453, 11, 262, 885, 1148, 233, 1149, 1551, 1936, 233, 4, 532, 6, 1937, 8, 97, 1938, 101, 30, 1939, 87], [132, 5, 208, 2, 39], [1552, 822, 465, 651, 161, 961, 214, 274, 1327, 105, 417, 215, 11, 1553, 58, 5, 10, 32, 323, 453, 234], [49, 8, 5, 2, 9, 4

## 2.5 Normalização

In [28]:
# Padding para garantir que todas as sequências tenham o mesmo comprimento
max_seq_length = max(len(seq) for seq in train_sequences)
train_padded = pad_sequences(train_sequences, maxlen=max_seq_length, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_seq_length, padding='post')

## 2.6 Categorizar as labels

In [29]:
# Codificação das labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

In [30]:
# Conversão para categórico
train_labels_categorical = to_categorical(train_labels_encoded)
test_labels_categorical = to_categorical(test_labels_encoded)

# 3. Word2Vec

Word2Vec é uma técnica de aprendizado de máquina que transforma palavras em vetores numéricos, capturando o significado semântico com base no contexto em que as palavras aparecem. Esses vetores permitem que palavras com significados semelhantes tenham representações semelhantes, facilitando o processamento de linguagem natural.

Word2Vec do NILC - Núcleo Interinstitucional de Linguística Computacional, na qual foi utilizado um WordEmbedding com 1000 dimensões.

In [31]:
Word2Vec = KeyedVectors.load_word2vec_format('/content/drive/Shareddrives/Grupo_05/cbow_s1000.txt')

In [ ]:
# # Função para criar o modelo com Word2Vec, camadas bidirecionais e Dropout
# def create_model(vocab_size, embedding_matrix, max_len):
#     model = Sequential()

#     # Embedding Layer com a matriz Word2Vec treinada
#     model.add(Embedding(input_dim=vocab_size,
#                         output_dim=embedding_matrix.shape[1],
#                         weights=[embedding_matrix],
#                         input_length=max_len,
#                         trainable=False))

#     # Camada LSTM bidirecional para capturar dependências em ambas as direções
#     model.add(Bidirectional(LSTM(128, return_sequences=True)))
#     model.add(GlobalMaxPooling1D())

#     # Camada Dropout para regularização
#     model.add(Dropout(0.5))

#     # Camada densa final para classificação
#     model.add(Dense(64, activation='relu'))
#     model.add(Dropout(0.5))  # Outra camada Dropout
#     model.add(Dense(3, activation='softmax'))  # Supondo que temos 3 classes de saída

#     # Compilar o modelo usando o otimizador Adam
#     model.compile(optimizer='adam',
#                   loss='categorical_crossentropy',
#                   metrics=['accuracy'])

#     return model

# # Função para EarlyStopping
# early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# # Funções de visualização de resultados

# # Função para plotar matriz de confusão
# def plot_confusion_matrix(y_true, y_pred, labels):
#     cm = confusion_matrix(y_true, y_pred)
#     plt.figure(figsize=(8, 6))
#     sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
#     plt.xlabel('Predicted')
#     plt.ylabel('Actual')
#     plt.title('Confusion Matrix')
#     plt.show()

# # Função para plotar Precision-Recall Curve
# def plot_precision_recall_curve(y_true, y_scores):
#     precision, recall, _ = precision_recall_curve(y_true, y_scores)
#     pr_auc = auc(recall, precision)

#     plt.plot(recall, precision, marker='.')
#     plt.xlabel('Recall')
#     plt.ylabel('Precision')
#     plt.title(f'Precision-Recall Curve (AUC = {pr_auc:.2f})')
#     plt.show()


In [41]:
def create_model(vocab_size, embedding_matrix, max_len):
    model = Sequential()

    # Embedding Layer com a matriz Word2Vec treinada
    model.add(Embedding(input_dim=vocab_size,
                        output_dim=embedding_matrix.shape[1],
                        weights=[embedding_matrix],
                        input_length=max_len,
                        trainable=False))

    # Primeira camada LSTM bidirecional
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(BatchNormalization())  # Adicionando BatchNormalization para estabilizar
    model.add(Dropout(0.5))  # Adicionando Dropout

    # Segunda camada LSTM bidirecional
    model.add(Bidirectional(LSTM(64, return_sequences=True)))
    model.add(GlobalMaxPooling1D())

    # Camada densa intermediária com Dropout
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))

    # Camada de saída com 3 classes
    model.add(Dense(3, activation='softmax'))

    # Compilar o modelo usando o otimizador Adam
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


# 4. Rede Neural Classificação

Nesta seção foi construída a rede neural para a classificação de intenção a partir das perguntas que foram fornecida na base de dados, tendo os seguintes componentes:

- Embedding: Converte palavras em vetores usando pesos pré-treinados do Word2Vec, mantendo as representações das palavras fixas.

- Bidirectional LSTM: Camadas LSTM bidirecionais que capturam dependências temporais no texto, considerando o contexto dos dois lados.

- Global Max Pooling: Reduz a dimensão dos vetores de saída das LSTMs, mantendo o valor máximo em cada dimensão.

- Densas e Dropout: Camadas densas para modelar relações complexas com ativação ReLU e Dropout para evitar overfitting.

- Saída Softmax: Gera probabilidades para cada classe, usando a função softmax.

 Já para a otimização foi utilizada a rmsprop, sendo um algoritmo que ajusta os pesos do modelo adaptando a taxa de aprendizado com base nas gradientes passadas, melhorando a convergência. Na perda categorical_crossentropy que mede a diferença entre as previsões do modelo e as classes verdadeiras em problemas de classificação múltipla. E para avaliação a acurácia que calcula a proporção de previsões corretas em relação ao total de previsões feitas pelo modelo.


## 4.1 Definição da rede neural

In [48]:
import matplotlib.pyplot as plt

def train_neural_network(train_padded, train_labels_categorical, vocab_size, embedding_matrix,
                         max_seq_length, output_classes, epochs=500, batch_size=128):
    """
    Treina um classificador de Rede Neural utilizando LSTM bidirecional e embeddings Word2Vec.

    Args:
        train_padded (np.array): Dados de treinamento com padding aplicado.
        train_labels_categorical (np.array): Labels de treinamento em formato categórico.
        vocab_size (int): Tamanho do vocabulário.
        embedding_matrix (np.array): Matriz de embeddings.
        max_seq_length (int): Tamanho máximo da sequência.
        output_classes (int): Número de classes de saída.
        epochs (int): Número de épocas para treinamento. Default é 500.
        batch_size (int): Tamanho do batch. Default é 128.

    Returns:
        model (Sequential): Modelo treinado.
        history (History): Histórico de treinamento.
        elapsed_time (float): Tempo decorrido durante o treinamento.
        memory_usage (float): Uso de memória durante o treinamento em MB.
    """
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=1000, weights=[embedding_matrix], input_length=max_seq_length, trainable=False),
        Bidirectional(LSTM(128)),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(output_classes, activation='softmax')
    ])

    # Compilar o modelo com RMSprop e categorical_crossentropy
    model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    # Callback EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    # Treinamento do modelo
    start_time = time.time()
    history = model.fit(train_padded, train_labels_categorical,
                        epochs=epochs, validation_data=(test_padded, test_labels_categorical),
                        batch_size=batch_size, callbacks=[early_stopping])
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Uso de memória
    process = psutil.Process()
    memory_usage = process.memory_info().rss / (1024 * 1024)  # Converter para MB

    # Plotar as métricas de treinamento
    plot_training_metrics(history)

    return model, history, elapsed_time, memory_usage


def plot_training_metrics(history):
    """
    Plota as métricas de perda e acurácia ao longo do treinamento.

    Args:
        history (History): Histórico de treinamento retornado pelo model.fit().
    """
    # Acurácia
    plt.figure(figsize=(12, 6))

    # Plotar acurácia de treino e validação
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Train vs Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plotar perda de treino e validação
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Train vs Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Mostrar o gráfico
    plt.tight_layout()  # Ajustar layout para não sobrepor
    plt.show()  # Certificar que o gráfico é mostrado


In [47]:
import matplotlib.pyplot as plt

def train_neural_network(train_padded, train_labels_categorical, vocab_size, embedding_matrix,
                         max_seq_length, output_classes, epochs=500, batch_size=128):
    """
    Treina um classificador de Rede Neural utilizando LSTM bidirecional e embeddings Word2Vec.

    Args:
        train_padded (np.array): Dados de treinamento com padding aplicado.
        train_labels_categorical (np.array): Labels de treinamento em formato categórico.
        vocab_size (int): Tamanho do vocabulário.
        embedding_matrix (np.array): Matriz de embeddings.
        max_seq_length (int): Tamanho máximo da sequência.
        output_classes (int): Número de classes de saída.
        epochs (int): Número de épocas para treinamento. Default é 500.
        batch_size (int): Tamanho do batch. Default é 128.

    Returns:
        model (Sequential): Modelo treinado.
        history (History): Histórico de treinamento.
        elapsed_time (float): Tempo decorrido durante o treinamento.
        memory_usage (float): Uso de memória durante o treinamento em MB.
    """
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=1000, weights=[embedding_matrix], input_length=max_seq_length, trainable=False),
        Bidirectional(LSTM(128)),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(output_classes, activation='softmax')
    ])

    # Compilar o modelo com RMSprop e categorical_crossentropy
    model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    # Callback EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    # Treinamento do modelo
    start_time = time.time()
    history = model.fit(train_padded, train_labels_categorical,
                        epochs=epochs, validation_data=(test_padded, test_labels_categorical),
                        batch_size=batch_size, callbacks=[early_stopping])
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Uso de memória
    process = psutil.Process()
    memory_usage = process.memory_info().rss / (1024 * 1024)  # Converter para MB

    # Plotar as métricas de treinamento
    plot_training_metrics(history)

    return model, history, elapsed_time, memory_usage


def plot_training_metrics(history):
    """
    Plota as métricas de perda e acurácia ao longo do treinamento.

    Args:
        history (History): Histórico de treinamento retornado pelo model.fit().
    """
    # Acurácia
    plt.figure(figsize=(12, 6))

    # Plotar acurácia de treino e validação
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Train vs Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plotar perda de treino e validação
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Train vs Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()


In [51]:
import matplotlib.pyplot as plt

# Garantir que o gráfico será exibido corretamente no Jupyter/Colab
%matplotlib inline

def train_neural_network(train_padded, train_labels_categorical, vocab_size, embedding_matrix,
                         max_seq_length, output_classes, epochs=500, batch_size=128):
    """
    Treina um classificador de Rede Neural utilizando LSTM bidirecional e embeddings Word2Vec.

    Args:
        train_padded (np.array): Dados de treinamento com padding aplicado.
        train_labels_categorical (np.array): Labels de treinamento em formato categórico.
        vocab_size (int): Tamanho do vocabulário.
        embedding_matrix (np.array): Matriz de embeddings.
        max_seq_length (int): Tamanho máximo da sequência.
        output_classes (int): Número de classes de saída.
        epochs (int): Número de épocas para treinamento. Default é 500.
        batch_size (int): Tamanho do batch. Default é 128.

    Returns:
        model (Sequential): Modelo treinado.
        history (History): Histórico de treinamento.
        elapsed_time (float): Tempo decorrido durante o treinamento.
        memory_usage (float): Uso de memória durante o treinamento em MB.
    """
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=1000, weights=[embedding_matrix], input_length=max_seq_length, trainable=False),
        Bidirectional(LSTM(128)),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(output_classes, activation='softmax')
    ])

    # Compilar o modelo com RMSprop e categorical_crossentropy
    model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    # Callback EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    # Treinamento do modelo
    start_time = time.time()
    history = model.fit(train_padded, train_labels_categorical,
                        epochs=epochs, validation_data=(test_padded, test_labels_categorical),
                        batch_size=batch_size, callbacks=[early_stopping])
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Uso de memória
    process = psutil.Process()
    memory_usage = process.memory_info().rss / (1024 * 1024)  # Converter para MB

    # Plotar as métricas de treinamento
    plot_training_metrics(history)

    return model, history, elapsed_time, memory_usage


def plot_training_metrics(history):
    """
    Plota as métricas de perda e acurácia ao longo do treinamento.

    Args:
        history (History): Histórico de treinamento retornado pelo model.fit().
    """
    # Acurácia
    plt.figure(figsize=(12, 6))

    # Plotar acurácia de treino e validação
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Train vs Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plotar perda de treino e validação
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Train vs Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Mostrar o gráfico
    plt.tight_layout()  # Ajustar layout para não sobrepor
    plt.show()  # Certificar que o gráfico é mostrado


In [52]:
def train_neural_network(train_padded, train_labels_categorical, test_padded, test_labels_categorical,
                         vocab_size, embedding_matrix, max_seq_length, output_classes, epochs=500, batch_size=128):
    """
    Treina um classificador de Rede Neural utilizando LSTM bidirecional e embeddings Word2Vec.

    Args:
        train_padded (np.array): Dados de treinamento com padding aplicado.
        train_labels_categorical (np.array): Labels de treinamento em formato categórico.
        test_padded (np.array): Dados de teste com padding aplicado.
        test_labels_categorical (np.array): Labels de teste em formato categórico.
        vocab_size (int): Tamanho do vocabulário.
        embedding_matrix (np.array): Matriz de embeddings.
        max_seq_length (int): Tamanho máximo da sequência.
        output_classes (int): Número de classes de saída.
        epochs (int): Número de épocas para treinamento. Default é 500.
        batch_size (int): Tamanho do batch. Default é 128.

    Returns:
        model (Sequential): Modelo treinado.
        history (History): Histórico de treinamento.
        elapsed_time (float): Tempo decorrido durante o treinamento.
        memory_usage (float): Uso de memória durante o treinamento em MB.
    """
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=1000, weights=[embedding_matrix], input_length=max_seq_length, trainable=False),
        Bidirectional(LSTM(128)),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(output_classes, activation='softmax')
    ])

    # Compilar o modelo com RMSprop e categorical_crossentropy
    model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    # Callback EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    # Treinamento do modelo
    start_time = time.time()
    history = model.fit(train_padded, train_labels_categorical,
                        epochs=epochs, validation_data=(test_padded, test_labels_categorical),
                        batch_size=batch_size, callbacks=[early_stopping])
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Uso de memória
    process = psutil.Process()
    memory_usage = process.memory_info().rss / (1024 * 1024)  # Converter para MB

    # Exibir métricas numéricas
    show_metrics(history)

    return model, history, elapsed_time, memory_usage


def show_metrics(history):
    """
    Exibe as métricas de perda e acurácia ao longo do treinamento.

    Args:
        history (History): Histórico de treinamento retornado pelo model.fit().
    """
    # Últimos valores de perda e acurácia de treino e validação
    final_train_accuracy = history.history['accuracy'][-1]
    final_val_accuracy = history.history['val_accuracy'][-1]
    final_train_loss = history.history['loss'][-1]
    final_val_loss = history.history['val_loss'][-1]

    # Exibir os resultados no console
    print("Final Training Accuracy: {:.4f}".format(final_train_accuracy))
    print("Final Validation Accuracy: {:.4f}".format(final_val_accuracy))
    print("Final Training Loss: {:.4f}".format(final_train_loss))
    print("Final Validation Loss: {:.4f}".format(final_val_loss))

    # Exibir as melhores métricas atingidas
    best_val_accuracy = max(history.history['val_accuracy'])
    best_val_loss = min(history.history['val_loss'])

    print("\nBest Validation Accuracy: {:.4f}".format(best_val_accuracy))
    print("Best Validation Loss: {:.4f}".format(best_val_loss))


In [53]:
from sklearn.metrics import precision_score, recall_score, f1_score

def train_neural_network(train_padded, train_labels_categorical, test_padded, test_labels_categorical,
                         vocab_size, embedding_matrix, max_seq_length, output_classes, epochs=500, batch_size=128):
    """
    Treina um classificador de Rede Neural utilizando LSTM bidirecional e embeddings Word2Vec.

    Args:
        train_padded (np.array): Dados de treinamento com padding aplicado.
        train_labels_categorical (np.array): Labels de treinamento em formato categórico.
        test_padded (np.array): Dados de teste com padding aplicado.
        test_labels_categorical (np.array): Labels de teste em formato categórico.
        vocab_size (int): Tamanho do vocabulário.
        embedding_matrix (np.array): Matriz de embeddings.
        max_seq_length (int): Tamanho máximo da sequência.
        output_classes (int): Número de classes de saída.
        epochs (int): Número de épocas para treinamento. Default é 500.
        batch_size (int): Tamanho do batch. Default é 128.

    Returns:
        model (Sequential): Modelo treinado.
        history (History): Histórico de treinamento.
        elapsed_time (float): Tempo decorrido durante o treinamento.
        memory_usage (float): Uso de memória durante o treinamento em MB.
    """
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=1000, weights=[embedding_matrix], input_length=max_seq_length, trainable=False),
        Bidirectional(LSTM(128)),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(output_classes, activation='softmax')
    ])

    # Compilar o modelo com RMSprop e categorical_crossentropy
    model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    # Callback EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    # Treinamento do modelo
    start_time = time.time()
    history = model.fit(train_padded, train_labels_categorical,
                        epochs=epochs, validation_data=(test_padded, test_labels_categorical),
                        batch_size=batch_size, callbacks=[early_stopping])
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Uso de memória
    process = psutil.Process()
    memory_usage = process.memory_info().rss / (1024 * 1024)  # Converter para MB

    # Exibir métricas numéricas de acurácia e perda
    show_metrics(history)

    # Previsões e métricas adicionais
    calculate_additional_metrics(model, test_padded, test_labels_categorical)

    return model, history, elapsed_time, memory_usage


def show_metrics(history):
    """
    Exibe as métricas de perda e acurácia ao longo do treinamento.

    Args:
        history (History): Histórico de treinamento retornado pelo model.fit().
    """
    # Últimos valores de perda e acurácia de treino e validação
    final_train_accuracy = history.history['accuracy'][-1]
    final_val_accuracy = history.history['val_accuracy'][-1]
    final_train_loss = history.history['loss'][-1]
    final_val_loss = history.history['val_loss'][-1]

    # Exibir os resultados no console
    print("Final Training Accuracy: {:.4f}".format(final_train_accuracy))
    print("Final Validation Accuracy: {:.4f}".format(final_val_accuracy))
    print("Final Training Loss: {:.4f}".format(final_train_loss))
    print("Final Validation Loss: {:.4f}".format(final_val_loss))

    # Exibir as melhores métricas atingidas
    best_val_accuracy = max(history.history['val_accuracy'])
    best_val_loss = min(history.history['val_loss'])

    print("\nBest Validation Accuracy: {:.4f}".format(best_val_accuracy))
    print("Best Validation Loss: {:.4f}".format(best_val_loss))


def calculate_additional_metrics(model, test_padded, test_labels_categorical):
    """
    Calcula e exibe as métricas Precision, Recall e F1-Score.

    Args:
        model (Sequential): Modelo treinado.
        test_padded (np.array): Dados de teste com padding aplicado.
        test_labels_categorical (np.array): Labels de teste em formato categórico.
    """
    # Previsões no conjunto de teste
    predictions = model.predict(test_padded)
    pred_labels = predictions.argmax(axis=1)
    true_labels = test_labels_categorical.argmax(axis=1)

    # Cálculo das métricas
    precision = precision_score(true_labels, pred_labels, average='weighted')
    recall = recall_score(true_labels, pred_labels, average='weighted')
    f1 = f1_score(true_labels, pred_labels, average='weighted')

    # Exibir as métricas de Precision, Recall e F1-Score
    print('\nMétricas Adicionais:')
    print(f'Precision: {precision:.2f}')
    print(f'Recall: {recall:.2f}')
    print(f'F1-Score: {f1:.2f}')
